## AMM Optimization, 2nd model (trading fees)

In the first experiment, we did not model the trading fees when executing a trade over a specific AMM. 

This approach is not realistic, as the trading fees are a significant part of the total cost of a trade for large trades. Moreover, in practice the pools on DEX are created by liquidity providers, who are incentivized to charge fees to the traders (in return for providing liquidity). In this experiment, we model the trading fees as a percentage of the trade size.

### 1. Problem formulation

Our approach for the modelization of the trading fees is derived from [Uniswap Whitepaper](https://hackmd.io/C-DvwDSfSxuh-Gd4WKE_ig).

let f be the trading fee for an AMM

I want to buy $ \Delta x $ ETH for $ \Delta y $ USDT 
If I send $ \Delta y $ USDT to the AMM, $f$% of it will be charged as a fee, so I will make the trade with $ \Delta y (1-f) $
The AMM will send me $ \Delta x $ ETH, and will receive $ \Delta y (1-f) $ USDT. Then, the invariant will be updated.

Therefore we have:

\begin{align} 
    (x_{eth} - \Delta x_{eth}) (y_{usd} + \Delta y_{usd}(1-f)) = x_{eth}y_{usd} \\
    \Delta y_{usd} = \frac{y_{usd} \Delta x_{eth}}{(1-f)(x_{eth} - \Delta x_{eth})} \\    
\end{align}



After that, the fee is added the the USDT pool, which makes the invariant change.

\begin{align} 
    x_{eth_{new}} = x_{eth} - \Delta x_{eth} \\
    y_{usd_{new}} = y_{usd} + \Delta y_{usd} \\
    invariant_{{new}} = x_{eth_{new}}y_{usd_{new}} 
\end{align}

So the quantity is bigger and the liquidity provider will receive some money afterwards


### 2. Implementation

In [1]:
using Gurobi, JuMP, CSV, Tables, Ipopt

In [18]:
# load data
#liquidity = CSV.read("../../data/ex_2_liquidity.csv", Tables.matrix)
#fees = CSV.read("../../data/ex_2_fees.csv", Tables.matrix)
#product = CSV.read("../../data/ex_2_k.csv", Tables.matrix); 

#liquidity = liquidity[1:size(fees, 1),:];

#println("Prices from the liquidity model", liquidity[:,2] ./ liquidity[:,1])

Prices from the liquidity model[690.7216494845361, 457.27956254272044, 5494.68085106383, 438.62376993212035, 66747.88758992805, 649.8326243455498, 1210.3490171311282, 520.789677764407, 412.8944702077082]


In [13]:
# With an order amount of token x
function optimize_price_paid_with_fees(liquidity, fees, order_amount)

    amount_token_x_per_pool = liquidity[:, 1] 
    amount_token_y_per_pool = liquidity[:, 2]

    # Create a new model
    model = Model(Ipopt.Optimizer)

    # Create variables
    @variable(model, x[1:size(liquidity,1)] >= 0)

    # Set objective
    @NLobjective(model, Min, sum((amount_token_y_per_pool[i] * x[i])/((1-fees[i])*(amount_token_x_per_pool[i]- x[i])) for i in 1:size(liquidity,1)))

    # Add constraint
    @constraint(model, sum(x) == order_amount)
    @constraint(model, x .<= amount_token_x_per_pool)

    # Optimize model
    optimize!(model)

    # Return the optimal solution
    return value.(x)
end

optimize_price_paid_with_fees (generic function with 1 method)

In [31]:
ETH_USDT = 1210;
# arrange for ETH
ETH = rand(3000:3500, 12)
ETH = sort(ETH, rev=false)
USD = ETH .* ETH_USDT
liquidity = [ETH USD];
fees = rand([0.3, 0.1, 0.01], 12)


println("ETH amount", liquidity[:, 1])
println("USDT amount", liquidity[:, 2])
println("Fees", fees)
order_amount = 1000
println("Order amount", order_amount)

split_trades = optimize_price_paid_with_fees(liquidity, fees, order_amount)

ETH amount[3023, 3068, 3100, 3117, 3256, 3261, 3263, 3292, 3413, 3430, 3454, 3476]
USDT amount[3657830, 3712280, 3751000, 3771570, 3939760, 3945810, 3948230, 3983320, 4129730, 4150300, 4179340, 4205960]
Fees[0.01, 0.01, 0.1, 0.01, 0.01, 0.01, 0.01, 0.3, 0.01, 0.01, 0.1, 0.3]
Order amount1000
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       12
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:       12

Total number of variables............................:       12
                     variables with only lower bounds:       12
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:       12
        inequality constraints with only lower bounds:     

12-element Vector{Float64}:
 117.02992528778981
 118.77201812203832
  -8.012587531206751e-9
 120.66896365266447
 126.05009485178728
 126.24366072225963
 126.32108707044821
  -9.893289338593107e-9
 132.12806318460935
 132.78618714421444
  -8.012596326102944e-9
  -9.893289339535232e-9

In [28]:
price = sum((split_trades[i]/order_amount)*liquidity[i, 2] / ((1-fees[i])*(liquidity[i, 1] - split_trades[i])) for i in 1:size(liquidity,1)) 
price_exec_baseline_pool = [liquidity[i, 2] / ((1-fees[i])*(liquidity[i, 1] - order_amount)) for i in 1:size(liquidity,1)]
avg_split = [order_amount / size(liquidity,1) for i in 1:size(liquidity,1)]
price_avg_amm = sum((avg_split[i]/order_amount)*liquidity[i, 2] / ((1-fees[i])*(liquidity[i, 1] - avg_split[i])) for i in 1:size(liquidity,1))
println("Price with split trades: ", price)
println("Price without split trades: ", price_exec_baseline_pool)
println("Price with even split on all Amms: ", price_avg_amm)

Price with split trades: -200.28403528042477
Price without split trades: [-200.28403525954946, -125.02998787724111, -193.54090354090354, -193.38545472258755, -124.08807967631498, -123.61814605360216, -123.55424644084438, -191.67920155148764, -122.85021973391126, -190.76804915514592, -189.36456063907045, -121.25071946586854]
Price with even split on all Amms: -113.28887272787341


##### Invariants analysis

In [29]:
del_y = [liquidity[i, 2]*split_trades[i]/ ((1-fees[i])*(liquidity[i, 1] - split_trades[i])) for i in 1:size(liquidity,1)]
println("Invariants before", [liquidity[i, 2] * liquidity[i, 1] for i in 1:size(liquidity,1)])
println("Invariants after", [(liquidity[i, 2] + del_y[i]) * (liquidity[i, 1] - split_trades[i]) for i in 1:size(liquidity,1)])



Invariants before[11197054440, 12694256410, 12978006250, 13025602810, 13137000250, 13369260960, 13401456640, 13571119210, 13766326090, 13880840490, 14385331840, 14670432040]
Invariants after[7.107254439550933e9, 1.2694256410041924e10, 1.297800625004394e10, 1.302560281004402e10, 1.3137000250042648e10, 1.3369260960043024e10, 1.3401456640043076e10, 1.3571119210044933e10, 1.376632609004366e10, 1.3880840490045443e10, 1.438533184004626e10, 1.467043204004507e10]


#### Market impact: TODO (not done) copied past from the first experiment (not adapted)

In [30]:
del_y = [liquidity[i, 2]*split_trades[i]/(liquidity[i, 1] - split_trades[i]) for i in 1:size(liquidity,1)]
price_post_trade_per_pool = [(liquidity[i, 2] + del_y[i])/ (liquidity[i, 1] - split_trades[i]) for i in 1:size(liquidity,1)]

# Comparison with an average split
avg_split = [order_amount / size(liquidity,1) for i in 1:size(liquidity,1)]
del_y_avg = [liquidity[i, 2]*avg_split[i]/(liquidity[i, 1] - avg_split[i]) for i in 1:size(liquidity,1)]
price_post_trade_per_pool_avg = [(liquidity[i, 2] + del_y_avg[i])/ (liquidity[i, 1] - avg_split[i]) for i in 1:size(liquidity,1)]

# Comparison with doing the trade on a single pool (each element in the array is "if we did the trade on this pool")
del_y_pool = [liquidity[i, 2]*order_amount/(liquidity[i, 1] - order_amount) for i in 1:size(liquidity,1)]
price_post_trade_per_pool_pool = [(liquidity[i, 2] + del_y_pool[i])/ (liquidity[i, 1] - order_amount) for i in 1:size(liquidity,1)]

println("Simulated price after trade per pool: ", price_post_trade_per_pool)
println("Simulated price after trade per pool if we had done an average split: ", price_post_trade_per_pool_avg)
println("Simulated price after trade per pool if we had done the trade on a single pool (displayed for all the pools): ", price_post_trade_per_pool_pool)

Simulated price after trade per pool: [2685.296923567897, 1209.9999999925406, 1209.9999999926258, 1209.9999999926395, 1209.9999999926672, 1209.9999999927313, 1209.99999999274, 1209.9999999927888, 1209.9999999928368, 1209.9999999928698, 1209.999999992996, 1209.999999993061]
Simulated price after trade per pool if we had done an average split: [1279.1212445630663, 1274.7500026471678, 1274.0102529842047, 1273.8886013180957, 1273.6065374223506, 1273.030112037498, 1272.9514238387712, 1272.541512682967, 1272.0795302236788, 1271.8131783528231, 1270.6788264810484, 1270.0644492313156]
Simulated price after trade per pool if we had done the trade on a single pool (displayed for all the pools): [2685.296923279111, 2532.206266315934, 2507.5244535684096, 2503.4980677348917, 2494.1974359339474, 2475.341671579359, 2472.78326897415, 2459.5158850418857, 2444.683133701963, 2436.188706491962, 2400.4742833952755, 2381.438385383626]
